# Common
Always run this, when start/restart the runtime

In [ ]:
import math
import torch
from torch import nn
from scipy import integrate
from threading import Thread

from PIL import Image
import numpy as np
from tqdm.auto import trange, tqdm

def append_dims(x, target_dims):
    """Appends dimensions to the end of a tensor until it has target_dims dimensions."""
    dims_to_append = target_dims - x.ndim
    if dims_to_append < 0:
        raise ValueError(f'input has {x.ndim} dims but target_dims is {target_dims}, which is less')
    return x[(...,) + (None,) * dims_to_append]


def append_zero(x):
    return torch.cat([x, x.new_zeros([1])])


def get_sigmas_karras(n, sigma_min, sigma_max, rho=7., device='cuda'):
    """Constructs the noise schedule of Karras et al. (2022)."""
    ramp = torch.linspace(0, 1, n,device=device)
    min_inv_rho = sigma_min ** (1 / rho)
    max_inv_rho = sigma_max ** (1 / rho)
    sigmas = (max_inv_rho + ramp * (min_inv_rho - max_inv_rho)) ** rho
    return append_zero(sigmas).to(device)


def get_sigmas_exponential(n, sigma_min, sigma_max, device='cpu'):
    """Constructs an exponential noise schedule."""
    sigmas = torch.linspace(math.log(sigma_max), math.log(sigma_min), n, device=device).exp()
    return append_zero(sigmas)


def get_sigmas_vp(n, beta_d=19.9, beta_min=0.1, eps_s=1e-3, device='cpu'):
    """Constructs a continuous VP noise schedule."""
    t = torch.linspace(1, eps_s, n, device=device)
    sigmas = torch.sqrt(torch.exp(beta_d * t ** 2 / 2 + beta_min * t) - 1)
    return append_zero(sigmas)


def to_d(x, sigma, denoised):
    """Converts a denoiser output to a Karras ODE derivative."""
    return (x - denoised) / append_dims(sigma, x.ndim)


def get_ancestral_step(sigma_from, sigma_to):
    """Calculates the noise level (sigma_down) to step down to and the amount
    of noise to add (sigma_up) when doing an ancestral sampling step."""
    sigma_up = (sigma_to ** 2 * (sigma_from ** 2 - sigma_to ** 2) / sigma_from ** 2) ** 0.5
    sigma_down = (sigma_to ** 2 - sigma_up ** 2) ** 0.5
    return sigma_down, sigma_up


@torch.no_grad()
def sample_euler(model, x, sigmas, extra_args=None, callback=None, disable=None, s_churn=0., s_tmin=0., s_tmax=float('inf'), s_noise=1.):
    """Implements Algorithm 2 (Euler steps) from Karras et al. (2022)."""
    extra_args = {} if extra_args is None else extra_args
    s_in = x.new_ones([x.shape[0]])
    for i in trange(len(sigmas) - 1, disable=disable):
        gamma = min(s_churn / (len(sigmas) - 1), 2 ** 0.5 - 1) if s_tmin <= sigmas[i] <= s_tmax else 0.
        eps = torch.randn_like(x) * s_noise
        sigma_hat = sigmas[i] * (gamma + 1)
        if gamma > 0:
            x = x + eps * (sigma_hat ** 2 - sigmas[i] ** 2) ** 0.5
        denoised = model(x, sigma_hat * s_in, **extra_args)
        d = to_d(x, sigma_hat, denoised)
        if callback is not None:
            callback({'x': x, 'i': i, 'sigma': sigmas[i], 'sigma_hat': sigma_hat, 'denoised': denoised})
        dt = sigmas[i + 1] - sigma_hat
        # Euler method
        x = x + d * dt
    return x


@torch.no_grad()
def sample_euler_ancestral(model, x, sigmas, extra_args=None, callback=None, disable=None):
    """Ancestral sampling with Euler method steps."""
    extra_args = {} if extra_args is None else extra_args
    s_in = x.new_ones([x.shape[0]])
    for i in trange(len(sigmas) - 1, disable=disable):
        denoised = model(x, sigmas[i] * s_in, **extra_args)
        sigma_down, sigma_up = get_ancestral_step(sigmas[i], sigmas[i + 1])
        if callback is not None:
            callback({'x': x, 'i': i, 'sigma': sigmas[i], 'sigma_hat': sigmas[i], 'denoised': denoised})
        d = to_d(x, sigmas[i], denoised)
        # Euler method
        dt = sigma_down - sigmas[i]
        x = x + d * dt
        x = x + torch.randn_like(x) * sigma_up
    return x


@torch.no_grad()
def sample_heun(model, x, sigmas, extra_args=None, callback=None, disable=None, s_churn=0., s_tmin=0., s_tmax=float('inf'), s_noise=1.):
    """Implements Algorithm 2 (Heun steps) from Karras et al. (2022)."""
    extra_args = {} if extra_args is None else extra_args
    s_in = x.new_ones([x.shape[0]])
    for i in trange(len(sigmas) - 1, disable=disable):
        gamma = min(s_churn / (len(sigmas) - 1), 2 ** 0.5 - 1) if s_tmin <= sigmas[i] <= s_tmax else 0.
        eps = torch.randn_like(x) * s_noise
        sigma_hat = sigmas[i] * (gamma + 1)
        if gamma > 0:
            x = x + eps * (sigma_hat ** 2 - sigmas[i] ** 2) ** 0.5
        denoised = model(x, sigma_hat * s_in, **extra_args)
        d = to_d(x, sigma_hat, denoised)
        if callback is not None:
            callback({'x': x, 'i': i, 'sigma': sigmas[i], 'sigma_hat': sigma_hat, 'denoised': denoised})
        dt = sigmas[i + 1] - sigma_hat
        if sigmas[i + 1] == 0:
            # Euler method
            x = x + d * dt
        else:
            # Heun's method
            x_2 = x + d * dt
            denoised_2 = model(x_2, sigmas[i + 1] * s_in, **extra_args)
            d_2 = to_d(x_2, sigmas[i + 1], denoised_2)
            d_prime = (d + d_2) / 2
            x = x + d_prime * dt
    return x


@torch.no_grad()
def sample_dpm_2(model, x, sigmas, extra_args=None, callback=None, disable=None, s_churn=0., s_tmin=0., s_tmax=float('inf'), s_noise=1.):
    """A sampler inspired by DPM-Solver-2 and Algorithm 2 from Karras et al. (2022)."""
    extra_args = {} if extra_args is None else extra_args
    s_in = x.new_ones([x.shape[0]])
    for i in trange(len(sigmas) - 1, disable=disable):
        gamma = min(s_churn / (len(sigmas) - 1), 2 ** 0.5 - 1) if s_tmin <= sigmas[i] <= s_tmax else 0.
        eps = torch.randn_like(x) * s_noise
        sigma_hat = sigmas[i] * (gamma + 1)
        if gamma > 0:
            x = x + eps * (sigma_hat ** 2 - sigmas[i] ** 2) ** 0.5
        denoised = model(x, sigma_hat * s_in, **extra_args)
        d = to_d(x, sigma_hat, denoised)
        if callback is not None:
            callback({'x': x, 'i': i, 'sigma': sigmas[i], 'sigma_hat': sigma_hat, 'denoised': denoised})
        # Midpoint method, where the midpoint is chosen according to a rho=3 Karras schedule
        sigma_mid = ((sigma_hat ** (1 / 3) + sigmas[i + 1] ** (1 / 3)) / 2) ** 3
        dt_1 = sigma_mid - sigma_hat
        dt_2 = sigmas[i + 1] - sigma_hat
        x_2 = x + d * dt_1
        denoised_2 = model(x_2, sigma_mid * s_in, **extra_args)
        d_2 = to_d(x_2, sigma_mid, denoised_2)
        x = x + d_2 * dt_2
    return x


@torch.no_grad()
def sample_dpm_2_ancestral(model, x, sigmas, extra_args=None, callback=None, disable=None):
    """Ancestral sampling with DPM-Solver inspired second-order steps."""
    extra_args = {} if extra_args is None else extra_args
    s_in = x.new_ones([x.shape[0]])
    for i in trange(len(sigmas) - 1, disable=disable):
        denoised = model(x, sigmas[i] * s_in, **extra_args)
        sigma_down, sigma_up = get_ancestral_step(sigmas[i], sigmas[i + 1])
        if callback is not None:
            callback({'x': x, 'i': i, 'sigma': sigmas[i], 'sigma_hat': sigmas[i], 'denoised': denoised})
        d = to_d(x, sigmas[i], denoised)
        # Midpoint method, where the midpoint is chosen according to a rho=3 Karras schedule
        sigma_mid = ((sigmas[i] ** (1 / 3) + sigma_down ** (1 / 3)) / 2) ** 3
        dt_1 = sigma_mid - sigmas[i]
        dt_2 = sigma_down - sigmas[i]
        x_2 = x + d * dt_1
        denoised_2 = model(x_2, sigma_mid * s_in, **extra_args)
        d_2 = to_d(x_2, sigma_mid, denoised_2)
        x = x + d_2 * dt_2
        x = x + torch.randn_like(x) * sigma_up
    return x


def linear_multistep_coeff(order, t, i, j):
    if order - 1 > i:
        raise ValueError(f'Order {order} too high for step {i}')
    def fn(tau):
        prod = 1.
        for k in range(order):
            if j == k:
                continue
            prod *= (tau - t[i - k]) / (t[i - j] - t[i - k])
        return prod
    return integrate.quad(fn, t[i], t[i + 1], epsrel=1e-4)[0]



def revpre0(img,sigmas,t):
  return img

def revpre1(img,sigmas,t):
  return (revpreimg+noise * sigmas[t])*(1-zamask)+img*zamask

revpre=revpre0

@torch.no_grad()
def sample_lms(model, x, sigmas, extra_args=None, callback=None, disable=None, order=4):
    extra_args = {} if extra_args is None else extra_args
    s_in = x.new_ones([x.shape[0]])
    ds = []
    for i in trange(len(sigmas) - 1, disable=disable):
        denoised = model( revpre(x,sigmas,i) , sigmas[i] * s_in, **extra_args)
        d = to_d(x, sigmas[i], denoised)
        ds.append(d)
        if len(ds) > order:
            ds.pop(0)
        if callback is not None:
            callback({'x': x, 'i': i, 'sigma': sigmas[i], 'sigma_hat': sigmas[i], 'denoised': denoised})
        cur_order = min(i + 1, order)
        coeffs = [linear_multistep_coeff(cur_order, sigmas.cpu(), i, j) for j in range(cur_order)]
        x = x + sum(coeff * d for coeff, d in zip(coeffs, reversed(ds)))
    return x


@torch.no_grad()
def log_likelihood(model, x, sigma_min, sigma_max, extra_args=None, atol=1e-4, rtol=1e-4):
    extra_args = {} if extra_args is None else extra_args
    s_in = x.new_ones([x.shape[0]])
    v = torch.randint_like(x, 2) * 2 - 1
    fevals = 0
    def ode_fn(sigma, x):
        nonlocal fevals
        with torch.enable_grad():
            x = x[0].detach().requires_grad_()
            denoised = model(x, sigma * s_in, **extra_args)
            d = to_d(x, sigma, denoised)
            fevals += 1
            grad = torch.autograd.grad((d * v).sum(), x)[0]
            d_ll = (v * grad).flatten(1).sum(1)
        return d.detach(), d_ll
    x_min = x, x.new_zeros([x.shape[0]])
    t = x.new_tensor([sigma_min, sigma_max])
    sol = odeint(ode_fn, x_min, t, atol=atol, rtol=rtol, method='dopri5')
    latent, delta_ll = sol[0][-1], sol[1][-1]
    ll_prior = torch.distributions.Normal(0, sigma_max).log_prob(latent).flatten(1).sum(1)
    return ll_prior + delta_ll, {'fevals': fevals}



class DiscreteSchedule(nn.Module):
    """A mapping between continuous noise levels (sigmas) and a list of discrete noise
    levels."""

    def __init__(self, sigmas, quantize):
        super().__init__()
        self.register_buffer('sigmas', sigmas)
        self.quantize = quantize

    def get_sigmas(self, n=None):
        if n is None:
            return append_zero(self.sigmas.flip(0))
        t_max = len(self.sigmas) - 1
        t = torch.linspace(t_max, 0, n, device=self.sigmas.device)
        return append_zero(self.t_to_sigma(t))

    def sigma_to_t(self, sigma, quantize=None):
        quantize = self.quantize if quantize is None else quantize
        
        dists = torch.abs(sigma - self.sigmas[:, None])
        if quantize:
            return torch.argmin(dists, dim=0).view(sigma.shape)
        low_idx, high_idx = torch.sort(torch.topk(dists, dim=0, k=2, largest=False).indices, dim=0)[0]
        low, high = self.sigmas[low_idx], self.sigmas[high_idx]
        w = (low - sigma) / (low - high)
        w = w.clamp(0, 1)
        t = (1 - w) * low_idx + w * high_idx
        return t.view(sigma.shape)

    def t_to_sigma(self, t):
        t = t.float()
        low_idx, high_idx, w = t.floor().long(), t.ceil().long(), t.frac()
        return (1 - w) * self.sigmas[low_idx] + w * self.sigmas[high_idx]


class DiscreteEpsDDPMDenoiser(DiscreteSchedule):
    """A wrapper for discrete schedule DDPM models that output eps (the predicted
    noise)."""

    def __init__(self, model, alphas_cumprod, quantize):
        super().__init__(((1 - alphas_cumprod) / alphas_cumprod) ** 0.5, quantize)
        self.inner_model = model
        self.sigma_data = 1.

    def get_scalings(self, sigma):
        c_out = -sigma
        c_in = 1 / (sigma ** 2 + self.sigma_data ** 2) ** 0.5
        return c_out, c_in

    def get_eps(self, *args, **kwargs):
        return self.inner_model(*args, **kwargs)

    def loss(self, input, noise, sigma, **kwargs):
        c_out, c_in = [append_dims(x, input.ndim) for x in self.get_scalings(sigma)]
        noised_input = input + noise * append_dims(sigma, input.ndim)
        eps = self.get_eps(noised_input * c_in, self.sigma_to_t(sigma), **kwargs)
        return (eps - noise).pow(2).flatten(1).mean(1)

    def forward(self, input, sigma, **kwargs):
        c_out, c_in = [append_dims(x, input.ndim) for x in self.get_scalings(sigma)]
        eps = self.get_eps(input * c_in, self.sigma_to_t(sigma), **kwargs)
        return input + eps * c_out



def make_ddim_timesteps(num_ddim_timesteps, num_ddpm_timesteps):
    c = num_ddpm_timesteps // num_ddim_timesteps
    ddim_timesteps = np.asarray(list(range(0, num_ddpm_timesteps, c)))

    # add one to get the final alpha values right (the ones from first scale to data during sampling)
    steps_out = ddim_timesteps + 1

    return steps_out


def make_ddim_sampling_parameters(alphacums, ddim_timesteps, eta):
    # select alphas for computing the variance schedule
    alphas = alphacums[ddim_timesteps]
    alphas_prev = np.asarray([alphacums[0]] + alphacums[ddim_timesteps[:-1]].tolist())

    # according the the formula provided in https://arxiv.org/abs/2010.02502
    sigmas = eta * np.sqrt((1 - alphas_prev) / (1 - alphas) * (1 - alphas / alphas_prev))

    return sigmas, alphas, alphas_prev

def makerng():
  if seed == 0:
    rng=random.randint(0, 2**32)
    np.random.seed(rng)
    torch.manual_seed(rng)
    print('random seed=')
    print(rng)
  else:
    np.random.seed(seed)
    torch.manual_seed(seed)

def dlpromptexample():
  !wget https://github.com/TabuaTambalam/DalleWebms/releases/download/0.1/pexmp.7z
  !7z x pexmp.7z
  

SDlatDEC=None
def latdec(fna):
  global SDlatDEC
  if SDlatDEC is None:
    if not os.path.isfile('autoencoder_pnnx.pt'):
      !wget https://huggingface.co/Larvik/sd470k/resolve/main/autoencoder_pnnx.pt
    SDlatDEC=torch.jit.load('autoencoder_pnnx.pt').cuda()
  lat=torch.tensor(np.load(fna)).cuda()
  return SDlatDEC(lat*5.5)


def localhttp(root='/'):
  global HTML
  if not os.path.isfile('/content/sample_data/izh.txt'):
    from IPython.core.display import HTML
    !nohup python3 -m http.server -d {root} 8233 > /content/sample_data/izh.txt &


class CompVisDenoiser(DiscreteEpsDDPMDenoiser):
    """A wrapper for CompVis diffusion models."""

    def __init__(self, model, quantize=False, device='cpu'):
        super().__init__(model, model.alphas_cumprod, quantize=quantize)

    def get_eps(self, *args, **kwargs):
        return self.inner_model.apply_model(*args, **kwargs)


class CFGDenoiser(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.inner_model = model

    def forward(self, x, sigma, uncond, cond, cond_scale):
        x_in = torch.cat([x] * 2)
        sigma_in = torch.cat([sigma] * 2)
        cond_in = torch.cat([uncond, cond])
        uncond, cond = self.inner_model(x_in, sigma_in, cond=cond_in).chunk(2)
        return uncond + (cond - uncond) * cond_scale


class SRDenoiser(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.inner_model = model

    def forward(self, x, sigma, cond ):
        cond = self.inner_model(x, sigma, cond=cond)
        return cond


class CompVisJIT():
  def __init__(self):
    self.alphas_cumprod=torch.tensor(alphas_cumprod,device=cudev)
    self.apply_model=apply_model

class ifeeder():
  def __init__(self):
    self.getn=self.get_simp
  def get_simp(self,n):
    return self.bs
  def setbs(self,in_bs):
    self.bs=in_bs
  
  def get_npbins(self,n):
    return torch.tensor(np.fromfile(self.pattern%(n+1),dtype=np.float32).reshape(self.shape),device=cudev)+self.noiseadd


model_wrap=None

In [ ]:
import os
if not os.path.isfile('MultiPromptExample1.txt'):
  t3 = Thread(target = dlpromptexample)
  a3 = t3.start()

Overwriting /content/MultiPromptExample1.txt


# Super Resolution 4x<br>
Select one of these task: Super Resolution, txt2img, (old ldm)infilling

In [ ]:
jit=False #@param {type:'boolean'}

import os

if not os.path.isfile('fsd_pnnx.pt'):
  !wget https://huggingface.co/Larvik/LDMjit/resolve/main/alphas_cumprod.npy
  !wget https://huggingface.co/Larvik/LDMjit/resolve/main/dm_pnnx.pt
  !wget https://huggingface.co/Larvik/LDMjit/resolve/main/fsd_pnnx.pt


import sys
import time

import numpy as np
import cv2
import functools
import torch

cudev=torch.device('cuda')

alphas_cumprod = np.load('alphas_cumprod.npy')


torch.set_grad_enabled(False)
torch.set_num_threads(os.cpu_count())
torch.backends.cudnn.enabled = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True


# ======================
# Arguemnt Parser Config
# ======================

def imread(filename, flags=cv2.IMREAD_COLOR):
    if not os.path.isfile(filename):
        print(f"File does not exist: {filename}")
        sys.exit()
    data = np.fromfile(filename, np.int8)
    img = cv2.imdecode(data, flags)
    return img

def preprocessing_img(img):
    if len(img.shape) < 3:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGRA)
    elif img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
    elif img.shape[2] == 1:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGRA)
    return img


def load_image(image_path):
    if os.path.isfile(image_path):
        img = imread(image_path, cv2.IMREAD_UNCHANGED)
    else:
        print(f'{image_path} not found.')
    return preprocessing_img(img)



def meshgrid(h, w):
    y = torch.arange(0, h).view(h, 1, 1).repeat(1, w, 1)
    x = torch.arange(0, w).view(1, w, 1).repeat(h, 1, 1)

    arr = torch.cat([y, x], dim=-1)
    return arr


def delta_border(h, w):
    """
    :param h: height
    :param w: width
    :return: normalized distance to image border,
      wtith min distance = 0 at border and max dist = 0.5 at image center
    """
    lower_right_corner = torch.tensor([h - 1, w - 1]).view(1, 1, 2)
    arr = meshgrid(h, w) / lower_right_corner
    dist_left_up = torch.min(arr, dim=-1, keepdims=True)[0]
    dist_right_down = torch.min(1 - arr, dim=-1, keepdims=True)[0]
    edge_dist = torch.min(torch.cat([dist_left_up, dist_right_down], dim=-1), dim=-1)[0]
    return edge_dist



def get_weighting(h, w, Ly, Lx, device):
  clip_min_weight = 0.01
  clip_max_weight = 0.5
  weighting = delta_border(h, w)
  weighting = torch.clip(weighting, clip_min_weight, clip_max_weight, )
  weighting = weighting.view(1, h * w, 1).repeat(1, 1, Ly * Lx).to(device)


  return weighting

def get_fold_unfold(x, kernel_size, stride, uf=1, df=1):  # todo load once not every time, shorten code
    """
    :param x: img of size (bs, c, h, w)
    :return: n img crops of size (n, bs, c, kernel_size[0], kernel_size[1])
    """
    bs, nc, h, w = x.shape

    # number of crops in image
    Ly = (h - kernel_size[0]) // stride[0] + 1
    Lx = (w - kernel_size[1]) // stride[1] + 1

    if uf == 1 and df == 1:
        fold_params = dict(kernel_size=kernel_size, dilation=1, padding=0, stride=stride)
        unfold = torch.nn.Unfold(**fold_params)

        fold = torch.nn.Fold(output_size=x.shape[2:], **fold_params)

        weighting = get_weighting(kernel_size[0], kernel_size[1], Ly, Lx, x.device).to(x.dtype)
        normalization = fold(weighting).view(1, 1, h, w)  # normalizes the overlap
        weighting = weighting.view((1, 1, kernel_size[0], kernel_size[1], Ly * Lx))

    elif uf > 1 and df == 1:
        fold_params = dict(kernel_size=kernel_size, dilation=1, padding=0, stride=stride)
        unfold = torch.nn.Unfold(**fold_params)

        fold_params2 = dict(kernel_size=(kernel_size[0] * uf, kernel_size[0] * uf),
                            dilation=1, padding=0,
                            stride=(stride[0] * uf, stride[1] * uf))
        fold = torch.nn.Fold(output_size=(x.shape[2] * uf, x.shape[3] * uf), **fold_params2)

        weighting = get_weighting(kernel_size[0] * uf, kernel_size[1] * uf, Ly, Lx, x.device).to(x.dtype)
        normalization = fold(weighting).view(1, 1, h * uf, w * uf)  # normalizes the overlap
        weighting = weighting.view((1, 1, kernel_size[0] * uf, kernel_size[1] * uf, Ly * Lx))

    elif df > 1 and uf == 1:
        fold_params = dict(kernel_size=kernel_size, dilation=1, padding=0, stride=stride)
        unfold = torch.nn.Unfold(**fold_params)

        fold_params2 = dict(kernel_size=(kernel_size[0] // df, kernel_size[0] // df),
                            dilation=1, padding=0,
                            stride=(stride[0] // df, stride[1] // df))
        fold = torch.nn.Fold(output_size=(x.shape[2] // df, x.shape[3] // df), **fold_params2)

        weighting = get_weighting(kernel_size[0] // df, kernel_size[1] // df, Ly, Lx, x.device).to(x.dtype)
        normalization = fold(weighting).view(1, 1, h // df, w // df)  # normalizes the overlap
        weighting = weighting.view((1, 1, kernel_size[0] // df, kernel_size[1] // df, Ly * Lx))

    else:
        raise NotImplementedError

    return fold, unfold, normalization, weighting



def normalize_image(image, normalize_type='255'):
    """
    Normalize image
    Parameters
    ----------
    image: numpy array
        The image you want to normalize
    normalize_type: string
        Normalize type should be chosen from the type below.
        - '255': simply dividing by 255.0
        - '127.5': output range : -1 and 1
        - 'ImageNet': normalize by mean and std of ImageNet
        - 'None': no normalization
    Returns
    -------
    normalized_image: numpy array
    """
    if normalize_type == 'None':
        return image
    elif normalize_type == '255':
        return image / 255.0
    elif normalize_type == '127.5':
        return image / 127.5 - 1.0
    elif normalize_type == 'ImageNet':
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = image / 255.0
        for i in range(3):
            image[:, :, i] = (image[:, :, i] - mean[i]) / std[i]
        return image
    else:
        pass



def preprocess(img):
    im_h, im_w, _ = img.shape

    up_f = 4
    oh, ow = up_f * im_h, up_f * im_w

    img = normalize_image(img, normalize_type='255')

    c = img * 2 - 1
    c = c.transpose(2, 0, 1)  # HWC -> CHW
    c = np.expand_dims(c, axis=0)
    c = c.astype(np.float32)

    return None, c


def postprocess(sample):
    sample = np.clip(sample, -1., 1.)
    sample = (sample + 1.) / 2. * 255
    sample = np.transpose(sample, (1, 2, 0))
    sample = sample[:, :, ::-1]  # RGB -> BGR
    sample = sample.astype(np.uint8)

    return sample


def decode_first_stage(z):
    ks = (128, 128)
    stride = (64, 64)
    uf = 4

    bs, nc, h, w = z.shape

    fold, unfold, normalization, weighting = get_fold_unfold(z, ks, stride, uf=uf)

    z = unfold(z)  # (bn, nc * prod(**ks), L)

    # Reshape to img shape
    z = z.view((z.shape[0], -1, ks[0], ks[1], z.shape[-1]))  # (bn, nc, ks[0], ks[1], L )


    print('first_stage_decode...')

    outputs = []
    for i in range(z.shape[-1]):
        x = z[:, :, :, :, i]
        output = first_stage_decode(x)
        outputs.append(output[0])

    o = torch.stack(outputs, axis=-1)  # # (bn, nc, ks[0], ks[1], L)
    o = o * weighting
    # Reverse 1. reshape to img shape
    o = o.view((o.shape[0], -1, o.shape[-1]))  # (bn, nc * ks[0] * ks[1], L)
    # stitch crops together
    decoded = fold(o)
    decoded = decoded / normalization  # norm is shape (1, 1, h, w)
    return decoded




# ddpm
def apply_model(x, t, cond):
    x_noisy=x
    ks = (128, 128)
    stride = (64, 64)

    h, w = x_noisy.shape[-2:]

    fold, unfold, normalization, weighting = get_fold_unfold(x_noisy, ks, stride)


    z = unfold(x_noisy)  # (bn, nc * prod(**ks), L)
    # Reshape to img shape
    z = z.view((z.shape[0], -1, ks[0], ks[1], z.shape[-1]))  # (bn, nc, ks[0], ks[1], L )
    z_list = [z[:, :, :, :, i] for i in range(z.shape[-1])]

    c = unfold(cond)
    c = c.view((c.shape[0], -1, ks[0], ks[1], c.shape[-1]))  # (bn, nc, ks[0], ks[1], L )
    cond_list = [c[:, :, :, :, i] for i in range(c.shape[-1])]

    # apply model by loop over crops
    
    outputs = []
    for i in range(z.shape[-1]):
        x = z_list[i]
        cond = cond_list[i]
        xc = torch.cat([x, cond], dim=1)
        
        
        output = diffusion_model(xc, t)
            
       
        outputs.append(output[0])

    o = torch.stack(outputs, axis=-1)
    o = o * weighting
    # Reverse reshape to img shape
    o = o.view((o.shape[0], -1, o.shape[-1]))  # (bn, nc * ks[0] * ks[1], L)
    # stitch crops together
    x_recon = fold(o) / normalization


    return x_recon

def warmup():
  v_0 = torch.rand(1,6,128,128, dtype=torch.float).half().cuda()
  v_1 = torch.randint(10, (1, ), dtype=torch.long).cuda()

  for d in range(2):
    with torch.cuda.amp.autocast(dtype=torch.float16):
      uaa = diffusion_model(v_0,v_1)
  v_0 = torch.rand(1,3,128,128, dtype=torch.float).cuda()
  for d in range(2):
    uaa = first_stage_decode(v_0)
  torch.cuda.empty_cache()


UseSamplr=sample_euler_ancestral
def predict(c):
    
    c=torch.tensor(c,device=cudev)

    sigmas = ddim_eta*model_wrap.get_sigmas(ddim_num_steps)  #ddim_sigmas #
    noise = torch.randn(c.shape, dtype=torch.float,device=cudev)
    
    extra_args = {'cond': c}
    df=detail_strength/(detail_strength-1+float(sigmas[0]))
    print(df)
    with torch.cuda.amp.autocast(dtype=torch.float16):
        samples = UseSamplr(model_wrap_cfg, noise * sigmas[0] * df , sigmas, extra_args=extra_args, disable=False)
   
    x_sample = decode_first_stage(samples)

    img = postprocess(x_sample[0].cpu().numpy())

    return img

if model_wrap is None:
  first_stage_decode=torch.jit.load('/content/fsd_pnnx.pt').eval().cuda()
  diffusion_model=torch.jit.load('/content/dm_pnnx.pt').eval().half().cuda()
  warmup()
  torch.cuda.empty_cache()
  model_wrap = CompVisDenoiser(CompVisJIT())
  model_wrap_cfg = SRDenoiser(model_wrap)

In [ ]:

image_path='/content/sample_data/10_0x0v1.png' #@param {type:'string'}

"""
ddim_timesteps
"""
ddim_eta = 0.75  #@param {type:'number'}
ddim_num_steps = 100  #@param {type:'number'}
ddpm_num_timesteps = 1000 #@param {type:'number'}

detail_strength=20000  #@param {type:'number'}

ddim_timesteps = make_ddim_timesteps(ddim_num_steps, ddpm_num_timesteps)




"""
ddim sampling parameters
"""

ddim_sigmas, ddim_alphas, ddim_alphas_prev = \
    make_ddim_sampling_parameters(
        alphacums=alphas_cumprod,
        ddim_timesteps=ddim_timesteps,
        eta=ddim_eta)

#ddim_sigmas=torch.tensor(ddim_sigmas.astype(np.float32),device=cudev)

ddim_sqrt_one_minus_alphas = np.sqrt(1. - ddim_alphas)



 




# inference
print('Start inference...')
if image_path.endswith('.npy'):
  c=latdec(image_path).detach()
else:
  img = load_image(image_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
  img = img[:, :, ::-1]  # BGR -> RGB
  _, c = preprocess(img)
  

img = predict(c)

# plot result
savepath = image_path[:-4]+'_4x.png'
print(f'saved at : {savepath}')
cv2.imwrite(savepath, img)



In [ ]:
'''
sample_euler
sample_euler_ancestral
sample_heun
sample_dpm_2
sample_dpm_2_ancestral
sample_lms
'''
UseSamplr = sample_dpm_2


Optional: SD lat decoder

In [ ]:
latent='4x6_1x1v1.npy' #@param {type:'string'}

ymg=Image.fromarray( (( ( latdec(latent)[0] +1)*127.5 ).cpu().numpy()).transpose(1,2,0).clip(0,255).astype(np.uint8) )
ymg.save(latent[:-4]+'.png')
ymg

# txt2img

In [ ]:
jit=False #@param {type:'boolean'}

import os

if not os.path.isfile('autoencoder_pnnx.pt'):
  !pip install transformers
  !wget https://huggingface.co/Larvik/sd470k/resolve/main/alphas_cumprod.npz
  !wget https://huggingface.co/Larvik/tfmod/resolve/main/transformer_pnnx.pt
  !wget https://huggingface.co/Larvik/sd470k/resolve/main/diffusion_emb_pnnx.pt
  !wget https://huggingface.co/Larvik/sd470k/resolve/main/diffusion_mid_pnnx.pt
  !wget https://huggingface.co/Larvik/sd470k/resolve/main/diffusion_out_pnnx.pt
  !wget https://huggingface.co/Larvik/sd470k/resolve/main/autoencoder_pnnx.pt
  !wget https://huggingface.co/Larvik/sd470k/resolve/main/imgencoder_pnnx.pt

import sys
import time
import random
import numpy as np
import cv2
from PIL import Image
import PIL
from transformers import CLIPTokenizer

import torch

torch.set_grad_enabled(False)
torch.set_num_threads(os.cpu_count())
torch.backends.cudnn.enabled = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True

alphas_cumprod = np.load('alphas_cumprod.npz')['a']

cudev=torch.device('cuda')



inzdict=[
'*',265,
'»',7599,
'¿',17133,
'¥',20199,
'®',8436]

class Insertor:
  def __init__(self, string, n):
    self.rpla=string
    self.rplb=inzdict[n]
    self.token=inzdict[n+1]
    self.idx=[]
    self.emb=torch.tensor(np.fromfile('UserEmb/'+string[1:-1]+'.bin',dtype=np.float32))
  def repl(self, string):
    return string.replace(self.rpla,self.rplb)


# encoder
class BERTEmbedder:
    """ Uses a pretrained BERT tokenizer by huggingface. Vocab size: 30522 (?)"""

    def __init__(self, transformer, max_length=77):
        self.tokenizer = CLIPTokenizer.from_pretrained('openai/clip-vit-large-patch14')
        self.max_length = max_length
        self.insertor=[]
        self.transformer = transformer
        self.embedding = torch.nn.Embedding.from_pretrained(self.transformer.state_dict()['text_model_embeddings_token_embedding.weight'])
        self.encode = self.encode0
        

    def insert(self,inz):
      if len(inz)<3:
        self.insertor=[]
        return
      n=len(self.insertor)
      if n > 4:
        return
      self.insertor.append(Insertor(inz,n*2))
      

    def encode0(self, text, nsamp):
        HasInz=False
        if self.insertor and len(text) > 0:
          HasInz=True
          for iz in self.insertor:
            text=iz.repl(text)
            iz.idx=[]
        batch_encoding = self.tokenizer(text, truncation=True, max_length=self.max_length, return_length=True,
                            return_overflowing_tokens=False, padding="max_length", return_tensors="pt")

        tokens = batch_encoding["input_ids"]
        
        if HasInz:
          for iz in self.insertor:
            iz.idx=torch.where(tokens == iz.token)[1]

        
        tokens = tokens.expand(nsamp,-1)

        amb=self.embedding(tokens)
        if HasInz:
          nl=amb.size(0)
          for iz in self.insertor:
            for pidx in iz.idx:
              for i in range(nl):
                amb[i][pidx]=iz.emb

        z = self.transformer(amb)
        return z.cuda()

    def encode2(self, text, nsamp):
        batch_encoding = self.tokenizer(text, truncation=True, max_length=self.max_length, return_length=True,
                            return_overflowing_tokens=False, padding="max_length", return_tensors="pt")

        tokens = batch_encoding["input_ids"]
        tokens = torch.tensor(tokens.numpy()).expand(nsamp,-1)

        z = self.transformer(tokens)
        return z.cuda()

    def encode3(self, text, nsamp):
      batch_encoding = self.tokenizer(text, truncation=True, max_length=self.max_length, return_length=True,
                            return_overflowing_tokens=False, padding="max_length", return_tensors="pt")

      return batch_encoding["input_ids"]

    def encode4(self, tokens):
       
         #.expand(ebb.size(0),1,-1) #.expand(1,77,-1)

        z = self.transformer(tokens)
        return z.cuda()





preimg=None
revpreimg=None




# ddpm
def apply_model(x, t, cond):

    h, emb, hs = diffusion_emb(x, t, cond)
    
    h = diffusion_mid(h, emb, cond, *hs[6:])

    output = diffusion_out(h, emb, cond, *hs[:6])

    return output


# decoder
def decode_first_stage(z):

    output = autoencoder(z/0.18215)
       
    return output


    
def load_img(path):
    image = Image.open(path).convert("RGB")
    w, h = image.size
    print(f"loaded input image of size ({w}, {h}) from {path}")
    w2, h2 = map(lambda x: x - x % 32, (w, h))
    if w!=w2 or h!=h2:
      image = image.resize((w2, h2), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

depthLimit=10

def txtErr(prmt0,msg):
  print(msg)
  prmt=prmt0.split('/')[-1][:-4]
  print('err prompt: '+prmt)
  return [cond_stage_model.encode(prmt, n_samples)]

def intptxtemb(stz,depth):
  prmpl=len(stz)>>1
  ptxt=[]
  pstp=[]
  for i in range(prmpl):
    ptxt.append(  makeCs(stz[2*i],depth)[0]  )
    pstp.append(  int(stz[2*i+1])+1  )
  prmpl-=1
  intpos=[]
  for vv in range(prmpl):
    c1=ptxt[vv]
    c2=ptxt[vv+1]
    stp=pstp[vv]
    for i in range(stp):
      intpos.append((c2*i+c1*(stp-i))/stp)

  lztbk=pstp[-1]
  if lztbk > 1:
    c1=ptxt[prmpl]
    c2=ptxt[0]
    for i in range(lztbk):
      intpos.append((c2*i+c1*(lztbk-i))/lztbk)
  else:
    intpos.append(ptxt[-1])
  return intpos


def encodepatt():
  ozi=output_pattern.split('/')[-1]
  pdir=output_pattern[:-len(ozi)-1]
  flist=os.listdir(pdir)
  flist.sort()
  pdir+='/'
  rpt=load_img(pdir+flist[0])
  vB=1
  vH=rpt.size(2)
  vW=rpt.size(3)
  thsize=torch.Size([vB,4,vH>>3,vW>>3])
  noyaz=torch.randn(thsize)
  zadd=0
  for f in flist:
    if f.endswith('.png'):
      vlat=imgenc(  load_img( pdir+f) ,  noyaz )*0.18215
      vlat.numpy().tofile(pdir+f[:-3]+'bin')
      zadd+=1
  with open(output_pattern[:-3].replace('%','!@!')+'txt','wt') as f:
    f.write(str(list(thsize))[1:-1]+'\n'+str(zadd))
  !rm {pdir}*.png





def makeCs(prmt,depth):
  if prmt.endswith('.txt'):
    if depth > depthLimit:
      return txtErr(prmt,'Too many ref, probably circular reference.')
    depth+=1
    if not os.path.isfile(prmt):
      return txtErr(prmt,'ref not found.')
    with open(prmt,'rt') as f:
      stz=f.read().splitlines()
    cmd=stz[0].replace(' ','').replace('\t','').split('/')
    cmd0=cmd[0]
    if cmd0.startswith('intp'):
      if depth > 1:
        return txtErr(stz[1],'do not intp in ref')
      return intptxtemb(stz[1:],depth)

    prmpl=(len(stz)-1)>>1
    stz=stz[1:]
    ptxt=[]
    pwgt=[]
    wgtsum=0
    for i in range(prmpl):
      ptxt.append(  makeCs(stz[2*i],depth)[0]  )
      wgt=float(stz[2*i+1])
      wgtsum+=wgt
      pwgt.append(  wgt  )
    if cmd0.startswith('avg'):
      for i in range(prmpl):
        pwgt[i]=pwgt[i]/wgtsum
    
    cout=ptxt[0]*pwgt[0]
    for i in range(1,prmpl):
      cout+=(ptxt[i]*pwgt[i])
    return [cout]
  else:
    return [cond_stage_model.encode(prmt, n_samples)]


def warmup():
  v_0 = torch.rand(2, 4, 32, 32, dtype=torch.float).half().cuda()
  v_1 = torch.randint(10, (2, ), dtype=torch.long).cuda()
  v_2 = torch.rand(2, 77, 768, dtype=torch.float).half().cuda()
  for d in range(2):
    with torch.cuda.amp.autocast(dtype=torch.float16):
      uaa = apply_model(v_0,v_1,v_2)
  v_0 = torch.rand(1, 4, 32, 32, dtype=torch.float).cuda()
  for d in range(2):
    uaa = autoencoder(v_0)
  torch.cuda.empty_cache()
  



fext='_%dx%dv%d.png'
def saver():
  global x_samples
  i=iita
  np.save( (outputp+fext%(i,1,ktta))[:-4] + '.npy', samples)
  x_samples = np.clip((x_samples.numpy() + 1.0) / 2.0, a_min=0.0, a_max=1.0)
  k=0
  for x_sample in x_samples:
      x_sample = x_sample.transpose(1, 2, 0)  # CHW -> HWC
      x_sample = x_sample * 255
      img = x_sample.astype(np.uint8)
      img = img[:, :, ::-1]  # RGB -> BGR
      cv2.imwrite(outputp+fext%(i,k,ktta), img)
      k+=1
  

UseSamplr=sample_lms

def predict(prompt, uc):
    global x_samples
    global samples
    global ktta
    global tmpfeeder
    global noise
    

    c_list = makeCs(prompt,0)
    feeder=ifeeder()
    
    sigmas = ddim_eta*model_wrap.get_sigmas(ddim_num_steps)
    noise = torch.randn(shape, dtype=torch.float,device=cudev)
    if preimg is not None:
      t_enc= int(strength * ddim_num_steps)
      sigma_sched = sigmas[ddim_num_steps - t_enc - 1:]
      if preimg.dim()==1:
        cmd0=int(preimg[0])
        if cmd0 == 2:
          feeder.pattern=tmpfeeder.pattern
          feeder.shape=tmpfeeder.shape
          feeder.getn=feeder.get_npbins
          feeder.noiseadd=noise * sigmas[ddim_num_steps - t_enc - 1]
          c_list=[c_list[0]]*tmpfeeder.xpenlen
          feeder.xpenlen=tmpfeeder.xpenlen
          tmpfeeder=feeder
      else:
        img = preimg.cuda() + noise * sigmas[ddim_num_steps - t_enc - 1]
        feeder.setbs(img)
    else:
      img = noise*sigmas[0]
      sigma_sched=sigmas
      feeder.setbs(img)


    ktta=0
    for c in c_list:
      extra_args = {'cond': c, 'uncond': uc, 'cond_scale': cfg_scale}
      with torch.cuda.amp.autocast(dtype=torch.float16):
        samples = UseSamplr(model_wrap_cfg, feeder.getn(ktta), sigma_sched, extra_args=extra_args, disable=False)
      ktta+=1
      
      
      x_samples = decode_first_stage(  samples ).cpu()
      samples=samples.cpu()
      t3 = Thread(target = saver)
      a3 = t3.start()

    return


def init_img_type():
  global init_img
  global tmpfeeder
  if init_img.endswith('.npy'):
    return 0
  elif init_img.endswith('.jpg') or init_img.endswith('.png'):
    if os.path.isfile(init_img+'.npy'):
      init_img+='.npy'
      return 0
    else:
      return 1
  elif init_img.endswith('.txt'):
    return 2
  else:
    return 99


if model_wrap is None:
  cond_stage_model = BERTEmbedder(torch.jit.load('transformer_pnnx.pt').eval())
  diffusion_emb = torch.jit.load('diffusion_emb_pnnx.pt').eval().half().cuda()
  diffusion_mid = torch.jit.load('diffusion_mid_pnnx.pt').eval().half().cuda()
  diffusion_out = torch.jit.load('diffusion_out_pnnx.pt').eval().half().cuda()
  autoencoder = torch.jit.load('autoencoder_pnnx.pt').eval().cuda()
  SDlatDEC=autoencoder
  imgenc = torch.jit.load('imgencoder_pnnx.pt').eval()
  warmup()
  torch.cuda.empty_cache()

  model_wrap = CompVisDenoiser(CompVisJIT())
  model_wrap_cfg = CFGDenoiser(model_wrap)

👇Optional👇

In [ ]:
'''
sample_euler
sample_euler_ancestral
sample_heun
sample_dpm_2
sample_dpm_2_ancestral
sample_lms
'''
UseSamplr = sample_heun

In [ ]:
cond_stage_model.insert('<>')
cond_stage_model.insert('<majipuri>')
cond_stage_model.insert('<pekora>')

In [ ]:
init_img='xxx' #@param {type:'string'}
strength=0.5 #@param {type:'number'}
initymgtyp=init_img_type()
if initymgtyp == 0:
  preimg=torch.tensor(np.load(init_img), device='cpu')
  n_samples=preimg.size(0)
  H=preimg.size(2)<<3
  W=preimg.size(3)<<3
elif initymgtyp == 1:
  n_samples=1
  rpt=load_img(init_img)
  H=rpt.size(2)
  W=rpt.size(3)
  preimg=imgenc(  rpt, torch.randn(torch.Size([n_samples,4,H>>3,W>>3]))  )*0.18215
  np.save(init_img+'.npy',preimg.numpy())
elif initymgtyp == 2:
  tmpfeeder=ifeeder()
  tmpfeeder.pattern=init_img[:-3].replace('!@!','%')+'bin'
  with open(init_img,'rt') as f:
    stz=f.read().replace(' ','').replace('\t','').splitlines()
  tmpfeeder.xpenlen=int(stz[1])
  stz=stz[0].split(',')
  tmpfeeder.shape=[ int(stz[0]), int(stz[1]), int(stz[2]), int(stz[3]) ]
  n_samples=tmpfeeder.shape[0]
  H=tmpfeeder.shape[2]<<3
  W=tmpfeeder.shape[3]<<3
  preimg=torch.tensor([2])
else:
  revpre=revpre0
  preimg=None
revpreimg=None

infilling

In [ ]:
FillFromNoise=False #@param {type:'boolean'}
masknpy='bench2_mask.npy' #@param {type:'string'}
zamask=torch.tensor(np.load(masknpy)).cuda()
revpreimg=preimg.cuda()
if FillFromNoise:
  preimg=None
revpre=revpre1

☝️Optional☝️

In [ ]:
InThread=False #@param {type:'boolean'}

prompt = 'a photograph of an astronaut riding a horse' #@param {type:'string'}

n_iter = 1 #@param {type:'integer'}
if preimg is None and revpreimg is None:
  n_samples = 1 #@param {type:'integer'}
  H=704 #@param {type:'integer'}
  W=768 #@param {type:'integer'}



ddim_num_steps = 50  #@param {type:'integer'}
ddpm_num_timesteps = 1000

seed=0 #@param {type:'integer'}

outputp='/content/sample_data' #@param {type:'string'}







cfg_scale = 7.5 #@param {type:'number'}
ddim_eta = 1.0  #@param {type:'number'}


outputp=outputp+'/'+str(len(os.listdir(outputp)))
"""
ddim_timesteps
"""

ddim_timesteps = make_ddim_timesteps(
    ddim_num_steps, ddpm_num_timesteps)

"""
ddim sampling parameters
"""

ddim_sigmas, ddim_alphas, ddim_alphas_prev = \
    make_ddim_sampling_parameters(
        alphacums=alphas_cumprod,
        ddim_timesteps=ddim_timesteps,
        eta=ddim_eta)

ddim_sqrt_one_minus_alphas = np.sqrt(1. - ddim_alphas)

shape = [n_samples, 4, H>>3 , W>>3 ]


makerng()


print("prompt: %s" % prompt)



print('Start inference...')
uc = None
if cfg_scale != 1.0:
  uc = cond_stage_model.encode('', n_samples)


  
def wpa():
  global iita
  torch.set_grad_enabled(False)
  
  for iita in range(n_iter):
      print("iteration: %s" % (iita + 1))
      predict(prompt, uc)
      
  print('Script finished successfully.')
  torch.cuda.empty_cache()


if InThread:
  t1 = Thread(target = wpa)
  a1 = t1.start()
else:
  wpa()


In [ ]:
!nvidia-smi

In [ ]:
!ffmpeg -framerate 3 -i /content/sample_data/48_0x3v%d.png intp03.mp4

# Tools
designed for the gen proc running with `InThread` or gradio app<br>
so imgenc (image->latent encoder) is on cpu

Gif/Video to latent pack

In [ ]:
input_anim  = '/content/senpai.gif' #@param {type:'string'}
output_pattern = '/content/ijj/senpai_%04d.png' #@param {type:'string'}
!ffmpeg -i {input_anim} {output_pattern}

Resize the output to `(64*n)x(64*m)` first

In [ ]:
encodepatt()

# Gradio Gui
tho I don't really understand why you want a webui inside another webui

In [ ]:
gradio=False #@param {type:'boolean'}

!pip install gradio
from google.colab import output
import gradio as gr

def dream():
  return


dream_interface = gr.Interface(
    dream,
    inputs=[
        gr.Textbox(placeholder="A corgi wearing a top hat as an oil painting.", lines=1),
        gr.Slider(minimum=1, maximum=150, step=1, label="Sampling Steps", value=50),
        gr.Checkbox(label='Enable PLMS sampling', value=False),
        gr.Checkbox(label='Enable Fixed Code sampling', value=False),
        gr.Slider(minimum=0.0, maximum=1.0, step=0.01, label="DDIM ETA", value=0.0, visible=False),
        gr.Slider(minimum=1, maximum=50, step=1, label='Sampling iterations', value=8),
        gr.Slider(minimum=1, maximum=8, step=1, label='Samples per iteration', value=1),
        gr.Slider(minimum=1.0, maximum=20.0, step=0.5, label='Classifier Free Guidance Scale', value=7.0),
        gr.Number(label='Seed', value=-1),
        gr.Slider(minimum=64, maximum=2048, step=64, label="Height", value=704),
        gr.Slider(minimum=64, maximum=2048, step=64, label="Width", value=768),
    ],
    outputs=[
        gr.Gallery(),
        gr.Number(label='Seed')
    ],
    title="Stable Diffusion Text-to-Image",
    description="Generate images from text with Stable Diffusion",
)


gdemo = gr.TabbedInterface(interface_list=[dream_interface], tab_names=["Dream"])


output.serve_kernel_port_as_window(8233, path='/dl.htm')

Copy the link above to `GoogleLocal`

In [ ]:
GoogleLocal = 'aaaaa' #@param {type:'string'}
if '.googleusercontent.com' in GoogleLocal:
  gdemo.launch()
else:
  print('set a valid GoogleLocal')